In [1]:
import yaml
import os
from collections import OrderedDict
from typing import Sequence, Dict, Optional, Sequence
from tqdm import tqdm
import torch
from torch.optim import Adam, SGD, RMSprop
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.utils import tensorboard

from models.classification import Classification
from classificationdataset import ClassificationDataset
from utils import *

## Load Task Config

In [2]:
configfile = open('config.yml')
config = yaml.load(configfile, Loader=yaml.FullLoader)

In [3]:
task = config['task']
taskconfig = config['tasks'][task.lower()]

In [4]:
print(f'The task defined is {task} with config {config}')


The task defined is classification with config {'task': 'classification', 'datadir': '/media/ranjan/dl/smartzoo/data', 'numepochs': 100, 'finetuneepochs': 0, 'logdir': 'logs', 'optimizer': {'name': 'rmsprop', 'alpha': 0.9, 'initlr': 0.256, 'momentum': 0.9, 'weightdecay': '1e-5'}, 'lossfn': 'crossentropyloss', 'pretrained': 'eff', 'pretrainedpath': 'logs/ckpt/ckpt_0.pt', 'batchsize': 4, 'imagesize': 300, 'tasks': {'classification': {'extractor': 'efficientnet-b3', 'numclasses': 6, 'weights': [], 'labelmapping': {'Lesser Mousedeer': 0, 'Long Tailed Macaque': 1, 'Others': 2, 'Sambar Deer': 3, 'Spotted Whistling Duck': 4, 'Wild Pig': 5}}}}


## Load Model

In [5]:
models = {
    'classification': Classification(extractor=taskconfig['extractor'], numclasses=taskconfig['numclasses'])
}

Loaded pretrained weights for efficientnet-b3


In [6]:
model = models[task]

In [7]:
startepoch = -1
if config['pretrained']  != 'imagenet' and config['pretrainedpath'] !='':
    print(f"Loading the checkpoint from {config['pretrainedpath']}")
    checkpoint = torch.load(config['pretrainedpath'], map_location='cpu')
    model.load_state_dict(checkpoint['model'])
    startepoch = checkpoint['epoch']

Loading the checkpoint from logs/ckpt/ckpt_0.pt


In [8]:
cudaavailable = torch.cuda.is_available()
device = torch.device('cuda') if cudaavailable else torch.device('cpu')
model.to(device)

Classification(
  (encoder): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          40, 10, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          10, 40, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
     

## Define loss function

In [9]:
lossfns = {
    'crossentropyloss': torch.nn.CrossEntropyLoss(reduction='none').to(device)
}

In [10]:
lossfn = lossfns[config['lossfn']]

## Define Optimizer

In [11]:
optimizers = {
    'sgd': SGD(model.parameters(), lr=float(config['optimizer']['initlr']), 
               momentum=float(config['optimizer']['momentum']), 
               weight_decay=float(config['optimizer']['momentum'])),
    'rmsprop': RMSprop(model.parameters(), lr=float(config['optimizer']['initlr']), 
                       alpha=float(config['optimizer']['alpha']),
                       momentum=float(config['optimizer']['momentum']), 
                       weight_decay=float(config['optimizer']['weightdecay']))
}

In [12]:
optimizer = optimizers[config['optimizer']['name']]
scheduler = lr_scheduler.StepLR(
            optimizer=optimizer, step_size=1, gamma=0.97 ** (1 / 2.4))

## Load Data

In [13]:
traindataset = ClassificationDataset(datadir=config['datadir'], fold='train', 
                                     imagesize=config['imagesize'], 
                                    labelmapping=taskconfig['labelmapping'])
traindataloader = DataLoader(traindataset, 
                             shuffle=True, 
                             pin_memory=True, 
                             num_workers=6, 
                             batch_size=config['batchsize'])

In [14]:
valdataset = ClassificationDataset(datadir=config['datadir'], fold='valid', 
                                   imagesize=config['imagesize'], 
                                   labelmapping=taskconfig['labelmapping'])
valdataloader = DataLoader(valdataset, 
                             shuffle=False, 
                             pin_memory=True, 
                             num_workers=6, 
                             batch_size=config['batchsize'])

## Create Tensorboard writer to write logs

In [15]:
logdir = config['logdir']

In [16]:
writer = tensorboard.SummaryWriter(config['logdir'])

## Train and Validate

In [17]:
def setfinetune(model: torch.nn.Module, finetune: bool) -> None:
    extractor = taskconfig['extractor']
    if finetune:
        finallayer = model.encoder.fc
        assert isinstance(finallayer, torch.nn.Module)

        #first set required grad to false
        model.encoder.requires_grad_(False)
        for param in model.encoder.fc.parameters():
            param.requires_grad = True
            
    else:
        model.encoder.requires_grad_(True)
    
        
    

In [18]:
def accuracy(outputs: torch.Tensor, labels: torch.Tensor,
            top: Sequence[int] = (1,)) -> Dict[int, float]:
    with torch.no_grad():
        # preds and labels both have shape [N, k]
        _, preds = outputs.topk(k=max(top), dim=1, largest=True, sorted=True)
        labels = labels.view(-1, 1).expand_as(preds)

        corrects = preds.eq(labels).cumsum(dim=1) 
        corrects = corrects.sum(dim=0)  # shape [k]
        
            
        tops = {k: corrects[k - 1].item() for k in top}
        
    return tops

In [19]:
def runepoch(model:torch.nn.Module, 
             dataloader: torch.utils.data.DataLoader,
             finetune: bool,
             device: torch.device,
             lossfn: Optional[torch.nn.Module] = None,
             optimizer: Optional[torch.nn.Module] = None,
             top: Sequence[int] = (1,)) -> Dict[str, float]:
    
    #set to eval when optimizer is none and also when finetune is True
    model.train(optimizer is not None and not finetune)
    
    #define object to track loss
    losses = Averagemeter()
    accuraciestopk = {k: Averagemeter() for k in top}
    
    tqdmloader = tqdm(dataloader)
    with torch.set_grad_enabled(optimizer is not None):
        for batch in tqdmloader:
            images, labels, imagepath = batch

            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            batchsize = labels.size(0)

            description = []
            outputs = model(images)

            if lossfn is not None:
                loss = lossfn(outputs, labels)
                loss = loss.mean()
                losses.update(loss.item(), batchsize)
                description.append(f'Loss {losses.val:.4f} ({losses.avg:.4f})')

            #backward pass
            if optimizer is not None:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
            topcorrect = accuracy(outputs, labels, top=top)
            for k, acc in accuraciestopk.items():
                acc.update(topcorrect[k] * (100. / batchsize), n=batchsize)
                description.append(f'accuracy@{k} {acc.val:.3f} ({acc.avg:.3f})')

            tqdmloader.set_description(' '.join(description))
            
    metrics = {}
    metrics['loss'] = losses.avg
    for k, acc in accuraciestopk.items():
        metrics[f'accuracytop{k}'] = acc.avg
        
    return metrics

In [20]:
bestmetrics: Dict[str, float] = {}
top = (1, 3)

In [ ]:
for epoch in range(int(config['numepochs']))[startepoch + 1:]:
    print(f'Epoch: {epoch + 1}')
    writer.add_scalar('lr', scheduler.get_last_lr()[0], epoch)
    
    #only finetune the final layers till the epochs less than finetuneepochs later tune entire model
    finetuneepochs = config['finetuneepochs']
    finetune = finetuneepochs > epoch
    setfinetune(model, finetune)
    
    
    print('Training')
    trainmetrics = runepoch(model, traindataloader, finetune, device, lossfn, optimizer, top)
    trainmetrics = prefix_all_keys(trainmetrics, prefix='train/')
    
    
    print('Validation')
    valmetrics = runepoch(model, valdataloader, finetune, device, lossfn, top=top)
    valmetrics = prefix_all_keys(valmetrics, prefix='val/')
    
    
    scheduler.step()
    
    if valmetrics['val/accuracytop1'] > bestmetrics.get('val/accuracytop1', 0):
        filename = os.path.join(logdir, 'ckpt', f'ckpt_{epoch}.pt')
        print(f'New best model! Saving checkpoint to {filename} with accuracy {valmetrics["val/accuracytop1"]}')
        state = {
            'epoch': epoch,
            'model': getattr(model, 'module', model).state_dict(),
            'val/acc': valmetrics['val/accuracytop1'],
            'optimizer': optimizer.state_dict()
        }
        torch.save(state, filename)
        bestmetrics.update(trainmetrics)
        bestmetrics.update(valmetrics)
        bestmetrics['epoch'] = epoch  

  0%|          | 0/2362 [00:00<?, ?it/s]

Epoch: 2
Training


Loss 3890.6895 (9865.4337) accuracy@1 50.000 (20.453) accuracy@3 100.000 (55.611): 100%|██████████| 2362/2362 [14:40<00:00,  2.68it/s]  
  0%|          | 0/1014 [00:00<?, ?it/s]

Validation


/home/ranjan/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/transforms/transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/ranjan/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/transforms/transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/ranjan/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/transforms/transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/ranjan/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/transforms/transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/ranjan

...

Loss 5168.3750 (5274.2330) accuracy@1 0.000 (19.680) accuracy@3 50.000 (56.077): 100%|██████████| 2362/2362 [22:15<00:00,  1.77it/s]  
  0%|          | 0/1014 [00:00<?, ?it/s]

Validation


/home/ranjan/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/transforms/transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/ranjan/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/transforms/transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/ranjan/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/transforms/transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/ranjan/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/transforms/transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/ranjan

Epoch: 24
Training


Loss 4196.8931 (4777.8994) accuracy@1 0.000 (18.944) accuracy@3 75.000 (56.171):  37%|███▋      | 871/2362 [08:16<14:38,  1.70it/s]  